<a href="https://colab.research.google.com/github/rahulhirur/FoundationStereo/blob/master/FoundationStereo_Collab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Install required libraries

In [1]:
!pip install scikit-image omegaconf opencv-contrib-python imgaug ninja timm albumentations jupyterlab scipy joblib scikit-learn ruamel.yaml trimesh pyyaml imageio open3d transformations einops gdown nodejs xformers==0.0.28.post1

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 6.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of torchvision to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 67.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 44.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.

Importing libraries

In [1]:
import os
import sys
import skimage
import logging
import torch
import imageio
import cv2
import numpy as np
import open3d as o3d
from omegaconf import OmegaConf
import plotly.graph_objects as go
import albumentations

Clone the Foundation stereo code

In [2]:
!git clone https://github.com/rahulhirur/FoundationStereo.git

Cloning into 'FoundationStereo'...
remote: Enumerating objects: 306, done.
remote: Counting objects: 100% (62/62), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 306 (delta 30), reused 8 (delta 3), pack-reused 244 (from 2)
Receiving objects: 100% (306/306), 16.18 MiB | 37.91 MiB/s, done.
Resolving deltas: 100% (86/86), done.


Change directories

In [3]:
%cd /content/FoundationStereo

/content/FoundationStereo


Create sub directory files

In [4]:
!mkdir pretrained_models/
!mkdir test_outputs/

In [5]:
code_dir = "/content/FoundationStereo"
out_dir = "/content/FoundationStereo/test_outputs"
ckpt_dir = "/content/FoundationStereo/pretrained_models/model_best_bp2.pth"

sys.path.append(f'{code_dir}/../')

config_file = f'{os.path.dirname(ckpt_dir)}/cfg.yaml'
run_config_file = f'{code_dir}/run_configuration.yaml'

import GPU based Foundation stereo libraries

In [6]:
from core.utils.utils import InputPadder
from Utils import *
from core.foundation_stereo import *

In [7]:
set_logging_format()
set_seed(0)
torch.autograd.set_grad_enabled(False)
os.makedirs(out_dir, exist_ok=True)

Copy paste pre-trained weights

In [8]:
!cp /content/drive/MyDrive/FoundationStereoImpModel/model_best_bp2.pth /content/FoundationStereo/pretrained_models
!cp /content/drive/MyDrive/FoundationStereoImpModel/cfg.yaml /content/FoundationStereo/pretrained_models

Read the config files

In [9]:
cfg = OmegaConf.load(config_file)
run_cfg = OmegaConf.load(run_config_file)
cfg = OmegaConf.merge(cfg, run_cfg)
args = OmegaConf.create(cfg)

In [10]:
logging.info(f"args:\n{args}")
logging.info(f"Using pretrained model from {ckpt_dir}")

args:
{'corr_implementation': 'reg', 'corr_levels': 2, 'corr_radius': 4, 'finetune_ckpt_name': 'model_best_bp2.pth', 'finetune_from': None, 'hidden_dims': [128, 128, 128], 'img_gamma': None, 'inference_tile': 0, 'low_memory': 0, 'max_disp': 416, 'max_val_sample': None, 'mixed_precision': True, 'n_downsample': 2, 'n_gru_layers': 3, 'notes': '', 'num_steps': 200000, 'num_worker': 8, 'slow_fast_gru': False, 'tags_more': [], 'tile_min_overlap': [16, 16], 'tile_wtype': 'gaussian', 'time_limit': 14400, 'train_iters': 22, 'val_interval': 1, 'valid_iters': 32, 'wdecay': 0, 'world_size': 32, 'left_file': './assets/left.png', 'right_file': './assets/right.png', 'intrinsic_file': './assets/K.txt', 'ckpt_dir': './pretrained_models/model_best_bp2.pth', 'out_dir': './output/', 'scale': 1.0, 'hiera': 0, 'z_far': 10.0, 'get_pc': 1, 'remove_invisible': 1, 'denoise_cloud': 0, 'denoise_nb_points': 30, 'denoise_radius': 0.03}
Using pretrained model from /content/FoundationStereo/pretrained_models/model_be

Model Initalization

In [11]:
model = FoundationStereo(args)
print('-------------------------1-------------------------')
# Check CUDA availability and load
if torch.cuda.is_available():
  ckpt = torch.load(ckpt_dir)
else:
  ckpt = torch.load(ckpt_dir, map_location=torch.device('cpu'))

logging.info(f"ckpt global_step:{ckpt['global_step']}, epoch:{ckpt['epoch']}")

Loading pretrained weights from Hugging Face hub (timm/edgenext_small.usi_in1k)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/22.4M [00:00<?, ?B/s]

[timm/edgenext_small.usi_in1k] Safe alternative available for 'pytorch_model.bin' (as 'model.safetensors'). Loading weights using safetensors.
/content/FoundationStereo/dinov2/layers/swiglu_ffn.py:43: UserWarning: xFormers is available (SwiGLU)
  warnings.warn("xFormers is available (SwiGLU)")
/content/FoundationStereo/dinov2/layers/attention.py:27: UserWarning: xFormers is available (Attention)
  warnings.warn("xFormers is available (Attention)")
/content/FoundationStereo/dinov2/layers/block.py:33: UserWarning: xFormers is available (Block)
  warnings.warn("xFormers is available (Block)")
Downloading: "https://github.com/facebookresearch/dinov2/zipball/main" to /root/.cache/torch/hub/main.zip
using MLP layer as FFN


-------------------------1-------------------------


<ipython-input-11-052518b40c77>:5: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(ckpt_dir)
ckpt global_step:200000, epoch:40


Load checkpoints

In [12]:
model.load_state_dict(ckpt['model'])

<All keys matched successfully>

Set model to eval mode

In [13]:
if torch.cuda.is_available():
  model.cuda()
else:
  model.cpu()
model.eval()

FoundationStereo(
  (cnet): ContextNetDino(
    (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3))
    (relu1): ReLU(inplace=True)
    (layer1): Sequential(
      (0): ResidualBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): ResidualBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (relu): ReLU(inplace=True)
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, 

Read Image

In [14]:
img0 = imageio.imread(args.left_file)
if img0.shape[-1] == 4:
  img0 = img0[:, :, :3]


img1 = imageio.imread(args.right_file)
if img1.shape[-1] == 4:
  img1 = img1[:, :, :3]

<ipython-input-14-5cc4fb3cb6cc>:1: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img0 = imageio.imread(args.left_file)
<ipython-input-14-5cc4fb3cb6cc>:6: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  img1 = imageio.imread(args.right_file)


Set Scale

In [15]:
scale = args.scale
assert scale<=1, "scale must be <=1"

In [16]:
img0 = cv2.resize(img0, fx=scale, fy=scale, dsize=None)
img1 = cv2.resize(img1, fx=scale, fy=scale, dsize=None)

In [17]:
H,W = img0.shape[:2]
img0_ori = img0.copy()
logging.info(f"img0: {img0.shape}")

img0: (540, 960, 3)


Convert imageio data to torch tensor

In [18]:
if torch.cuda.is_available():
  img0 = torch.as_tensor(img0).cuda().float()[None].permute(0,3,1,2)
  img1 = torch.as_tensor(img1).cuda().float()[None].permute(0,3,1,2)
else:
  img0 = torch.as_tensor(img0).float()[None].permute(0,3,1,2)
  img1 = torch.as_tensor(img1).float()[None].permute(0,3,1,2)

padder = InputPadder(img0.shape, divis_by=32, force_square=False)
img0, img1 = padder.pad(img0, img1)

Run Model

In [19]:
if torch.cuda.is_available(): #checks if cuda is available

    with torch.cuda.amp.autocast(True):

        if not args.hiera:
            disp = model.forward(img0, img1, iters=args.valid_iters, test_mode=True)
        else:
            disp = model.run_hierachical(img0, img1, iters=args.valid_iters, test_mode=True, small_ratio=0.5)
else: #if cuda is not available, run on cpu

    if not args.hiera:
        print("enterng inside the model forward")
        disp = model.forward(img0, img1, iters=args.valid_iters, test_mode=True)

    else:
        disp = model.run_hierachical(img0, img1, iters=args.valid_iters, test_mode=True, small_ratio=0.5)


<ipython-input-19-8316b71c07c5>:3: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(True):
/content/FoundationStereo/core/foundation_stereo.py:204: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.args.mixed_precision):


---------------1--------------
---------------2--------------
------------------F1---------------------
------------------F2---------------------
------------------F3---------------------
------------------F4---------------------
------------------F5---------------------
------------------F6---------------------
---------------3--------------


/content/FoundationStereo/core/submodule.py:395: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):


---------------FeatAtt_Vastr--------------
---------------FeatAtt_Vastr--------------
---------------FeatAtt_Vastr--------------
---------------FeatAtt_Vastr--------------
---------------FeatAtt_Vastr--------------
---------------FeatAtt_Vastr--------------
---------------4--------------
---------------5--------------


/content/FoundationStereo/core/geometry.py:74: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
/content/FoundationStereo/core/foundation_stereo.py:248: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.args.mixed_precision):


---------------6--------------
---------------7--------------


/content/FoundationStereo/core/foundation_stereo.py:185: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=self.args.mixed_precision):


Save Disparity

In [26]:
disp = padder.unpad(disp.float())
disp = disp.data.cpu().numpy().reshape(H,W)
vis = vis_disparity(disp)
vis = np.concatenate([img0_ori, vis], axis=1)
imageio.imwrite(f'{out_dir}/vis.png', vis)
logging.info(f"Output saved to {args.out_dir}")

AttributeError: 'numpy.ndarray' object has no attribute 'float'

Remove invisible section

In [27]:
if args.remove_invisible:
  yy,xx = np.meshgrid(np.arange(disp.shape[0]), np.arange(disp.shape[1]), indexing='ij')
  us_right = xx-disp
  invalid = us_right<0
  disp[invalid] = np.inf

Generate point cloud

In [30]:
if args.get_pc:
  with open(args.intrinsic_file, 'r') as f:
    lines = f.readlines()
    K = np.array(list(map(float, lines[0].rstrip().split()))).astype(np.float32).reshape(3,3)
    baseline = float(lines[1])
  K[:2] *= scale
  depth = K[0,0]*baseline/disp
  np.save(f'{out_dir}/depth_meter.npy', depth)
  xyz_map = depth2xyzmap(depth, K)
  pcd = toOpen3dCloud(xyz_map.reshape(-1,3), img0_ori.reshape(-1,3))
  keep_mask = (np.asarray(pcd.points)[:,2]>0) & (np.asarray(pcd.points)[:,2]<=args.z_far)
  keep_ids = np.arange(len(np.asarray(pcd.points)))[keep_mask]
  pcd = pcd.select_by_index(keep_ids)
  o3d.io.write_point_cloud(f'{out_dir}/cloud.ply', pcd)
  logging.info(f"PCL saved to {out_dir}")

PCL saved to /content/FoundationStereo/test_outputs


Denoise point cloud

In [31]:
if args.denoise_cloud:
  logging.info("denoise point cloud...")
  cl, ind = pcd.remove_radius_outlier(nb_points=args.denoise_nb_points, radius=args.denoise_radius)
  inlier_cloud = pcd.select_by_index(ind)
  o3d.io.write_point_cloud(f'{args.out_dir}/cloud_denoise.ply', inlier_cloud)
  pcd = inlier_cloud

read point cloud and visualize using plotly

In [35]:
cloud = o3d.io.read_point_cloud(f'{out_dir}/cloud.ply')
if cloud.is_empty(): print('error in reading ply file')
points = np.asarray(cloud.points)
colors = None
if cloud.has_colors():
    colors = np.asarray(cloud.colors)
elif cloud.has_normals():
    colors = (0.5, 0.5, 0.5) + np.asarray(cloud.normals) * 0.5
else:
    geometry = o3d.geometry.PointCloud()
    geometry.points = cloud.points
    geometry.paint_uniform_color((1.0, 0.0, 0.0))
    colors = np.asarray(geometry.colors)

In [ ]:
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=points[:,0], y=points[:,1], z=points[:,2],
            mode='markers',
            marker=dict(size=1, color=colors)
        )
    ],
    layout=dict(
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False)
        )
    )
)
fig.show()

'/content/FoundationStereo/test_outputs'